In [2]:
import json

from langchain.chains.question_answering import load_qa_chain

from langchain_community.llms import Ollama
from langchain_community.embeddings.laser import LaserEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatMaritalk

from langchain_core.prompts.chat import ChatPromptTemplate

from langchain.text_splitter import RecursiveCharacterTextSplitter

import numpy as np

import pandas as pd


In [ ]:
# obtains the API key used to access Maritalk's API
with open('API_KEY.json', 'r') as file:
    maritalk = json.load(file)

model = ChatMaritalk(
    model='sabia-3',
    api_key=maritalk['key'],
    temperature=0.7,
    max_tokens=500,
)

In [3]:
ollama_base_url= 'http://tempestade.facom.ufms.br:11435' #'http://localhost:11434' 
model = Ollama(base_url=ollama_base_url, model="phi3:medium")

# embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [4]:
embeddings_model = LaserEmbeddings(lang='por_Latn')

2024-09-23 07:24:21,203 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-09-23 07:24:22,014 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2024-09-23 07:24:22,180 | INFO | laser_encoders.download_models |  - laser2.pt already downloaded
2024-09-23 07:24:22,182 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2024-09-23 07:24:22,182 | INFO | laser_encoders.download_models |  - laser2.cvocab already downloaded


In [ ]:
documents_dir_path = './database'

loader = DirectoryLoader(documents_dir_path, glob='./*.pdf', loader_cls=PyPDFLoader)

loaded_pdfs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=0,
        separators=['\n', ' ']
)

pages = text_splitter.split_documents(loaded_pdfs)

In [ ]:
vectorstore = FAISS.from_documents(
    pages,
    embeddings_model
)

vectorstore.save_local('database/indexes/document_index')

In [5]:
# loads vectorstore from disk
vectorstore = FAISS.load_local('database/indexes/document_index', embeddings_model, allow_dangerous_deserialization=True)

2024-09-23 07:25:14,066 | INFO | faiss.loader | Loading faiss with AVX2 support.
2024-09-23 07:25:14,735 | INFO | faiss.loader | Successfully loaded faiss with AVX2 support.


In [6]:
prompt_structure = '''
        Baseado nos seguintes documentos:
        {context}
        Responda a pergunta abaixo:
        {query}
        '''

qa_prompt = ChatPromptTemplate.from_messages([
        ('system', '''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Use a seguinte estrutura para responder as perguntas:'''),
        ('human', '''Como será a lista de espera?'''),
        ('ai', '''De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.'''),
        ('human', '''Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?'''),
        ('ai', '''De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.'''),
        ('human', prompt_structure)
])

chain = load_qa_chain(model, chain_type='stuff', verbose=True, prompt=qa_prompt)

In [ ]:
query = input()

In [10]:
retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10)

In [ ]:
# enriching the retrieval result 

# get the page index of the page with the most similar chunk
page_idx = retrieval_result[0][0].metadata['page']

retrieval_result = np.concatenate(
    (
        [x for x in pages if x.metadata['page'] == page_idx],
        [page[0] for page in retrieval_result]),
    axis=0
)

In [11]:
# retrieved context, no enrichment
retrieval_result= [page[0] for page in retrieval_result]

In [12]:
output = chain.invoke(
        {'input_documents':retrieval_result, 'query':query}
)

print(output['output_text'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Use a seguinte estrutura para responder as perguntas:
Human: Como será a lista de espera?
AI: De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.
Human: Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?
AI: De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.
Human: 
        Baseado nos seguintes documentos:
        7.1. Os inscritos serão classiﬁcados d

### Geração das respostas

In [14]:
queries = pd.read_csv('tests/test_questions.csv', dtype={'numero':int})

enrichment = False

answers = []

for num_doc in [70, 9]:

    questions = queries[queries['numero'] == num_doc]['pergunta'].tolist()

    for question in questions:
        
        query = f'De acordo com o edital {num_doc}. ' + question

        retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10)

        if enrichment:
            # enriching the retrieval result 

            # get the page index of the page with the most similar chunk
            page_idx = retrieval_result[0][0].metadata['page']

            retrieval_result = np.concatenate(
                (
                    [x for x in pages if x.metadata['page'] == page_idx],
                    [page[0] for page in retrieval_result]),
                axis=0
            )
        else:
            # retrieved context, no enrichment
            retrieval_result= [page[0] for page in retrieval_result]

        output = chain.invoke(
            {'input_documents':retrieval_result, 'query':query}
        )

        answers.append(output['output_text'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Use a seguinte estrutura para responder as perguntas:
Human: Como será a lista de espera?
AI: De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.
Human: Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?
AI: De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.
Human: 
        Baseado nos seguintes documentos:
        7.1. Os inscritos serão classiﬁcados d

In [22]:
result = pd.DataFrame(
    {
        'pergunta':queries['pergunta'].tolist(),
        'resposta':answers,
        'edital':queries['arquivo'].tolist()
    }
)

In [23]:
# modelo: phi3 medium
# enriquecimento do contexto: não
# chain of thought: sim
result

,pergunta,resposta,edital
0,Quem pode receber a Bolsa Permanência do MEC?,"De acordo com o Edital, os beneficiários da Bo...",edital-1
1,Qual o período de inscrições para a Bolsa Perm...,"De acordo com o Edital, não há informações esp...",edital-1
2,Quais documentos são necessários para a inscri...,"De acordo com os artigos 6 e 11.2 do Edital, o...",edital-1
3,Onde os estudantes devem se inscrever para a B...,"De acordo com o artigo 5 deste Edital, os insc...",edital-1
4,Quais são os motivos para a suspensão da Bolsa...,"De acordo com o item 6.1 do Edital, um estudan...",edital-1
5,Quantas bolsas estão disponíveis para estudant...,"De acordo com o Edital, existem setenta (70) v...",edital-1
6,Como é definida a lista de espera?,"De acordo com o item 3.4 do Edital, a lista de...",edital-1
7,Quando serão divulgados os resultados?,"De acordo com o Edital nº 8323, artigo 5, item...",edital-1
8,Qual é o objetivo do Edital PROAES/PROGRAD/PRO...,O objetivo do Edital Proaes/Prograd/Propp/UFMS...,edital-2
9,Quais são os valores das bolsas oferecidas par...,"De acordo com o Edital, os valores da Bolsa Pe...",edital-2


In [24]:
result.to_csv('tests/results/phi3-no-enrichment-tuned-prompt.csv', index=False)